In [ ]:
pip install pyspark


In [ ]:
pip install vnstock3 --upgrade

In [ ]:
import datetime
import pandas as pd
from vnstock import stock_historical_data
from pyspark.sql import SparkSession

# Tạo SparkSession
spark = SparkSession.builder \
    .appName("Stock Data Processing") \
    .getOrCreate()

# Danh sách mã cổ phiếu lớn từ 20 ngành
stock_symbols = [
    "VCB", "VNM", "MWG", "VIC", "SSI", "DGC", "CTD", "FPT", "MSN",
    "GVR", "GAS", "POW", "HPG", "REE", "DHG", "GMD", "VHC", "KBC",
    "CMG", "VRE"
]

# Bước 1: Lấy dữ liệu chứng khoán từ Vnstock API
today = datetime.date.today()
four_years_ago = today - datetime.timedelta(days=1461)  # 4 năm

# Tạo một danh sách để lưu trữ dữ liệu
all_data = []

for stock_symbol in stock_symbols:
    # Lấy dữ liệu từ API
    data = stock_historical_data(
        stock_symbol,
        start_date=four_years_ago.strftime('%Y-%m-%d'),
        end_date=today.strftime('%Y-%m-%d')
    )

    # Điều chỉnh dữ liệu
    df_pandas = pd.DataFrame(data)[['time', 'open', 'close', 'high', 'low', 'volume']]
    df_pandas = df_pandas.rename(columns={'time': 'date'})

    # Thêm cột 'ticker'
    df_pandas['ticker'] = stock_symbol

    # Thêm dữ liệu vào danh sách
    all_data.append(df_pandas)

# Kết hợp tất cả các dữ liệu vào một DataFrame duy nhất
df_combined = pd.concat(all_data, ignore_index=True)

# Chuyển đổi sang Spark DataFrame
df_spark = spark.createDataFrame(df_combined)

# Hiển thị schema của DataFrame
df_spark.printSchema()

# Xuất kết quả ra DataFrame và hiển thị
result_df = df_spark.toPandas()
result_df

# Dừng SparkSession
spark.stop()

In [ ]:
result_df

In [ ]:
import datetime
import pandas as pd
from vnstock import stock_historical_data
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import plotly.graph_objs as go
import plotly.subplots as sp
import ipywidgets as widgets
from IPython.display import display, clear_output

# Khởi tạo dictionary để lưu trữ mô hình và các chỉ số hiệu suất
models = {}

# Danh sách các mã cổ phiếu
stock_symbols = ["VCB", "VNM", "MWG", "VIC", "SSI", "DGC", "CTD", "FPT", "MSN",
    "GVR", "GAS", "POW", "HPG", "REE", "DHG", "GMD", "VHC", "KBC",
    "CMG", "VRE"]


# Bước 1: Lấy dữ liệu cổ phiếu từ API Vnstock
today = datetime.date.today()
four_years_ago = today - datetime.timedelta(days=1461)  # 4 năm

for stock_symbol in stock_symbols:
    # Lấy dữ liệu từ API
    data = stock_historical_data(
        stock_symbol,
        start_date=four_years_ago.strftime('%Y-%m-%d'),
        end_date=today.strftime('%Y-%m-%d')
    )

    if data.empty:  # Kiểm tra xem DataFrame có rỗng không
        continue  # Bỏ qua nếu không có dữ liệu

    # Điều chỉnh dữ liệu
    df_pandas = pd.DataFrame(data)[['time', 'close']]
    df_pandas = df_pandas.rename(columns={'time': 'date'})
    df_pandas['date'] = pd.to_datetime(df_pandas['date'])
    df_pandas = df_pandas.sort_values(by='date')

    # Chuẩn bị dữ liệu
    prices = df_pandas['close'].values.reshape(-1, 1)

    # Chuẩn hóa dữ liệu
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(prices)

    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    train_size = int(len(scaled_data) * 0.7)
    train_data = scaled_data[:train_size, :]
    test_data = scaled_data[train_size:, :]

    # Tạo cặp đầu vào-đầu ra cho mô hình
    def create_dataset(dataset, look_back=5):
        X, y = [], []
        for i in range(len(dataset) - look_back):
            X.append(dataset[i:(i + look_back), 0])
            y.append(dataset[i + look_back, 0])
        return np.array(X), np.array(y)

    X_train, y_train = create_dataset(train_data)
    X_test, y_test = create_dataset(test_data)

    # Định hình lại dữ liệu đầu vào cho LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Xây dựng mô hình LSTM
    model = Sequential()
    model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(5, 1)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(50)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Huấn luyện mô hình
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=2, validation_data=(X_test, y_test), callbacks=[early_stopping])

    # Dự đoán và đánh giá hiệu suất
    y_pred = model.predict(X_test)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_inv = scaler.inverse_transform(y_pred)

    # Tính toán các chỉ số hiệu suất
    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r_squared = r2_score(y_test_inv, y_pred_inv)
    mape = np.mean(np.abs((y_test_inv - y_pred_inv) / y_test_inv)) * 100

    # Lưu mô hình và các chỉ số vào dictionary
    models[stock_symbol] = {
        'model': model,
        'y_test': y_test_inv.flatten(),
        'y_pred': y_pred_inv.flatten(),
        'dates': df_pandas['date'].values[-len(y_test):],
        'rmse': rmse,
        'mae': mae,
        'r_squared': r_squared,
        'mape': mape
    }



In [ ]:
# Hàm vẽ biểu đồ với các chỉ báo kỹ thuật
def plot_technical_indicators(stock_symbol):
    today = datetime.date.today()
    one_year_ago = today - datetime.timedelta(days=1461)

    # Lấy dữ liệu từ API
    data = stock_historical_data(
        stock_symbol,
        start_date=one_year_ago.strftime('%Y-%m-%d'),
        end_date=today.strftime('%Y-%m-%d')
    )

    # Chuẩn bị dữ liệu
    df = pd.DataFrame(data)[['time', 'open', 'close', 'high', 'low', 'volume']]
    df = df.rename(columns={'time': 'date'})
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')

    # Bỏ các ngày không giao dịch
    df = df.dropna()

    # Chuyển ngày thành chuỗi để dùng trục categorical
    df['date_str'] = df['date'].dt.strftime('%Y-%m-%d')

    # Tính các chỉ báo kỹ thuật
    df['MA50'] = df['close'].rolling(window=50).mean()
    df['MA100'] = df['close'].rolling(window=100).mean()
    df['BB_upper'] = df['close'].rolling(window=20).mean() + 2 * df['close'].rolling(window=20).std()
    df['BB_lower'] = df['close'].rolling(window=20).mean() - 2 * df['close'].rolling(window=20).std()
    df['MACD'] = df['close'].ewm(span=12).mean() - df['close'].ewm(span=26).mean()
    df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()

    # Tính RSI
    delta = df['close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # Tạo biểu đồ với Nến Nhật và Khối Lượng trên cùng biểu đồ
    fig = sp.make_subplots(
        rows=3, cols=1, shared_xaxes=True,
        row_heights=[0.6, 0.2, 0.2], vertical_spacing=0.05,
        specs=[[{"secondary_y": True}], [{}], [{}]]  # Khối lượng trên trục Y phụ
    )

    # Nến Nhật
    fig.add_trace(
        go.Candlestick(
            x=df['date_str'], open=df['open'], high=df['high'],
            low=df['low'], close=df['close'], name='Candlestick'
        ), row=1, col=1, secondary_y=False
    )

    # Khối lượng giao dịch trên trục Y phụ
    fig.add_trace(
        go.Bar(
            x=df['date_str'], y=df['volume'],
            marker_color='lightblue', opacity=0.5, name='Volume'
        ), row=1, col=1, secondary_y=True
    )

    # MA50 và MA100
    fig.add_trace(go.Scatter(x=df['date_str'], y=df['MA50'], mode='lines', name='MA50'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['date_str'], y=df['MA100'], mode='lines', name='MA100'), row=1, col=1)

    # Bollinger Bands
    fig.add_trace(go.Scatter(x=df['date_str'], y=df['BB_upper'], line=dict(color='lightgray'), name='BB Upper'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['date_str'], y=df['BB_lower'], line=dict(color='lightgray'), name='BB Lower'), row=1, col=1)

    # MACD và Signal Line
    fig.add_trace(go.Scatter(x=df['date_str'], y=df['MACD'], mode='lines', name='MACD'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df['date_str'], y=df['MACD_Signal'], mode='lines', name='MACD Signal'), row=2, col=1)

    # RSI
    fig.add_trace(go.Scatter(x=df['date_str'], y=df['RSI'], mode='lines', name='RSI'), row=3, col=1)

    # Vùng quá mua và quá bán cho RSI
    fig.add_shape(type='line', x0=df['date_str'].min(), x1=df['date_str'].max(), y0=70, y1=70,
                  line=dict(color='red', dash='dash'), row=3, col=1)
    fig.add_shape(type='line', x0=df['date_str'].min(), x1=df['date_str'].max(), y0=30, y1=30,
                  line=dict(color='green', dash='dash'), row=3, col=1)

    # Tùy chỉnh bố cục với hai trục Y
    fig.update_layout(
        title=f'{stock_symbol} - Technical Indicators and Volume',
        template='plotly_white',
        height=1000, width=1200,
        hovermode='x unified',  # Hover đồng bộ
        xaxis=dict(
            type='category',  # Loại bỏ khoảng trống
            tickangle=45  # Xoay nhãn để dễ đọc
        ),
        yaxis=dict(
            title='Price',  # Trục Y chính cho giá
            side='left'
        ),
        yaxis2=dict(
            title='Volume',  # Trục Y phụ cho khối lượng
            overlaying='y',  # Chồng lên trục Y chính
            side='right'
        ),
        xaxis_rangeslider_visible=False  # Ẩn thanh cuộn
    )

    # Hiển thị biểu đồ
    fig.show()



In [ ]:
from keras.layers import GRU, Dropout, Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Tạo dictionary để lưu trữ mô hình GRU và các chỉ số hiệu suất
gru_models = {}

for stock_symbol in stock_symbols:
    # Lấy dữ liệu từ API
    data = stock_historical_data(stock_symbol, start_date=four_years_ago.strftime('%Y-%m-%d'), end_date=today.strftime('%Y-%m-%d'))

    if data.empty:
        continue

    # Điều chỉnh dữ liệu
    df_pandas = pd.DataFrame(data)[['time', 'close']]
    df_pandas = df_pandas.rename(columns={'time': 'date'})
    df_pandas['date'] = pd.to_datetime(df_pandas['date'])
    df_pandas = df_pandas.sort_values(by='date')

    # Chuẩn bị dữ liệu
    prices = df_pandas['close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(prices)

    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    train_size = int(len(scaled_data) * 0.7)
    train_data = scaled_data[:train_size, :]
    test_data = scaled_data[train_size:, :]

    # Tạo cặp đầu vào-đầu ra cho mô hình
    X_train, y_train = create_dataset(train_data)
    X_test, y_test = create_dataset(test_data)

    # Định hình lại dữ liệu đầu vào cho GRU
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Xây dựng mô hình GRU
    gru_model = Sequential()
    gru_model.add(GRU(128, return_sequences=True, input_shape=(X_train.shape[1], 1)))  # Tăng số lượng đơn vị
    gru_model.add(Dropout(0.3))  # Tăng dropout
    gru_model.add(GRU(64))  # Tăng số lượng đơn vị
    gru_model.add(Dropout(0.3))
    gru_model.add(Dense(1))
    gru_model.compile(loss='mean_squared_error', optimizer='adam')

    # Định nghĩa các callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

    # Huấn luyện mô hình
    gru_model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=2, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr])

    # Dự đoán và đánh giá hiệu suất
    y_pred_gru = gru_model.predict(X_test)
    y_test_inv_gru = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_inv_gru = scaler.inverse_transform(y_pred_gru)

    # Tính toán các chỉ số hiệu suất
    rmse_gru = np.sqrt(mean_squared_error(y_test_inv_gru, y_pred_inv_gru))
    mae_gru = mean_absolute_error(y_test_inv_gru, y_pred_inv_gru)
    r_squared_gru = r2_score(y_test_inv_gru, y_pred_inv_gru)
    mape_gru = np.mean(np.abs((y_test_inv_gru - y_pred_inv_gru) / y_test_inv_gru)) * 100

    # Lưu mô hình GRU và các chỉ số vào dictionary
    gru_models[stock_symbol] = {
        'model': gru_model,
        'y_test': y_test_inv_gru.flatten(),
        'y_pred': y_pred_inv_gru.flatten(),
        'dates': df_pandas['date'].values[-len(y_test):],
        'rmse': rmse_gru,
        'mae': mae_gru,
        'r_squared': r_squared_gru,
        'mape': mape_gru
    }


In [ ]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Dense, BatchNormalization
from keras.models import Sequential
from keras.callbacks import EarlyStopping

# Tạo dictionary để lưu trữ mô hình CNN và các chỉ số hiệu suất
cnn_models = {}

for stock_symbol in stock_symbols:
    # Lấy dữ liệu từ API
    data = stock_historical_data(stock_symbol, start_date=four_years_ago.strftime('%Y-%m-%d'), end_date=today.strftime('%Y-%m-%d'))

    if data.empty:
        continue

    # Điều chỉnh dữ liệu
    df_pandas = pd.DataFrame(data)[['time', 'close']]
    df_pandas = df_pandas.rename(columns={'time': 'date'})
    df_pandas['date'] = pd.to_datetime(df_pandas['date'])
    df_pandas = df_pandas.sort_values(by='date')

    # Chuẩn bị dữ liệu
    prices = df_pandas['close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(prices)

    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    train_size = int(len(scaled_data) * 0.7)
    train_data = scaled_data[:train_size, :]
    test_data = scaled_data[train_size:, :]

    # Tạo cặp đầu vào-đầu ra cho mô hình
    X_train, y_train = create_dataset(train_data)
    X_test, y_test = create_dataset(test_data)

    # Định hình lại dữ liệu đầu vào cho CNN
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Xây dựng mô hình CNN với cải tiến
    cnn_model = Sequential()
    cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(X_train.shape[1], 1)))
    cnn_model.add(BatchNormalization())
    cnn_model.add(MaxPooling1D(pool_size=2))
    cnn_model.add(Dropout(0.2))

    cnn_model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    cnn_model.add(BatchNormalization())
    cnn_model.add(MaxPooling1D(pool_size=2))
    cnn_model.add(Dropout(0.2))

    cnn_model.add(Flatten())
    cnn_model.add(Dense(50, activation='relu'))
    cnn_model.add(Dense(1))  # Đầu ra là một giá trị duy nhất
    cnn_model.compile(loss='mean_squared_error', optimizer='adam')

    # Cài đặt Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Huấn luyện mô hình
    cnn_model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=2, validation_data=(X_test, y_test), callbacks=[early_stopping])

    # Dự đoán và đánh giá hiệu suất
    y_pred_cnn = cnn_model.predict(X_test)
    y_test_inv_cnn = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_inv_cnn = scaler.inverse_transform(y_pred_cnn)

    # Tính toán các chỉ số hiệu suất
    rmse_cnn = np.sqrt(mean_squared_error(y_test_inv_cnn, y_pred_inv_cnn))
    mae_cnn = mean_absolute_error(y_test_inv_cnn, y_pred_inv_cnn)
    r_squared_cnn = r2_score(y_test_inv_cnn, y_pred_inv_cnn)
    mape_cnn = np.mean(np.abs((y_test_inv_cnn - y_pred_inv_cnn) / y_test_inv_cnn)) * 100

    # Lưu mô hình CNN và các chỉ số vào dictionary
    cnn_models[stock_symbol] = {
        'model': cnn_model,
        'y_test': y_test_inv_cnn.flatten(),
        'y_pred': y_pred_inv_cnn.flatten(),
        'dates': df_pandas['date'].values[-len(y_test):],
        'rmse': rmse_cnn,
        'mae': mae_cnn,
        'r_squared': r_squared_cnn,
        'mape': mape_cnn
    }


In [ ]:
# Hàm hiển thị biểu đồ kỹ thuật
def display_technical_chart(stock_symbol):
    clear_output(wait=True)
    print(f"Hiển thị biểu đồ kỹ thuật cho cổ phiếu: {stock_symbol}")
    plot_technical_indicators(stock_symbol)
    display(reset_button)

# Hàm hiển thị biểu đồ dự đoán
def display_prediction_chart(stock_symbol):
    clear_output(wait=True)
    print(f"Hiển thị dữ liệu dự đoán cho cổ phiếu: {stock_symbol}")

    fig = go.Figure()
    metrics_summary = {}

    # Vẽ biểu đồ dự đoán cho LSTM
    if stock_symbol in models:
        y_test_lstm = models[stock_symbol]['y_test']
        y_pred_lstm = models[stock_symbol]['y_pred']
        dates = models[stock_symbol]['dates']

        fig.add_trace(go.Scatter(x=dates, y=y_test_lstm, mode='lines', name='Giá thực tế', line=dict(color='blue', width=2, shape='spline')))
        fig.add_trace(go.Scatter(x=dates, y=y_pred_lstm, mode='lines', name='Giá dự đoán LSTM', line=dict(color='orange', shape='spline')))

        # Lưu chỉ số đánh giá của LSTM
        metrics_summary['LSTM'] = {
            'rmse': models[stock_symbol]['rmse'],
            'mae': models[stock_symbol]['mae'],
            'r_squared': models[stock_symbol]['r_squared'],
            'mape': models[stock_symbol]['mape']
        }

    # Vẽ biểu đồ dự đoán cho GRU
    if stock_symbol in gru_models:
        y_test_gru = gru_models[stock_symbol]['y_test']
        y_pred_gru = gru_models[stock_symbol]['y_pred']
        dates_gru = gru_models[stock_symbol]['dates']

        fig.add_trace(go.Scatter(x=dates_gru, y=y_pred_gru, mode='lines', name='Giá dự đoán GRU', line=dict(color='red', dash='dash', shape='spline')))

        # Lưu chỉ số đánh giá của GRU
        metrics_summary['GRU'] = {
            'rmse': gru_models[stock_symbol]['rmse'],
            'mae': gru_models[stock_symbol]['mae'],
            'r_squared': gru_models[stock_symbol]['r_squared'],
            'mape': gru_models[stock_symbol]['mape']
        }

    # Vẽ biểu đồ dự đoán cho CNN
    if stock_symbol in cnn_models:
        y_test_cnn = cnn_models[stock_symbol]['y_test']
        y_pred_cnn = cnn_models[stock_symbol]['y_pred']
        dates_cnn = cnn_models[stock_symbol]['dates']

        fig.add_trace(go.Scatter(x=dates_cnn, y=y_pred_cnn, mode='lines', name='Giá dự đoán CNN', line=dict(color='green', dash='dot', shape='spline')))

        # Lưu chỉ số đánh giá của CNN
        metrics_summary['CNN'] = {
            'rmse': cnn_models[stock_symbol]['rmse'],
            'mae': cnn_models[stock_symbol]['mae'],
            'r_squared': cnn_models[stock_symbol]['r_squared'],
            'mape': cnn_models[stock_symbol]['mape']
        }

    # Cập nhật layout biểu đồ
    fig.update_layout(title=f'Dự đoán giá cổ phiếu {stock_symbol}',
                      xaxis_title='Ngày',
                      yaxis_title='Giá',
                      template='plotly_white')
    fig.show()

    # Hiển thị các chỉ số đánh giá
    for model_name, metrics in metrics_summary.items():
        print(f"Dự đoán và Giá thực tế ({model_name}):")
        print(f"RMSE ({model_name}): {metrics['rmse']:.2f}")
        print(f"MAE ({model_name}): {metrics['mae']:.2f}")
        print(f"R² ({model_name}): {metrics['r_squared']:.2f}")
        print(f"MAPE ({model_name}): {metrics['mape']:.2f}%")
        print()

    # So sánh hiệu suất của các mô hình
    best_model = min(metrics_summary.items(), key=lambda x: x[1]['rmse'])[0]
    print(f"\033[1mMô hình hiệu quả nhất là: {best_model} với RMSE: {metrics_summary[best_model]['rmse']:.2f}\033[0m")

    display(reset_button)

# Tạo widget dropdown
stock_dropdown = widgets.Dropdown(
    options=stock_symbols,
    description='Mã cổ phiếu:',
    value=stock_symbols[0]  # Giá trị mặc định
)

# Tạo nút để hiển thị biểu đồ kỹ thuật
tech_button = widgets.Button(description='Hiển thị biểu đồ kỹ thuật', layout=widgets.Layout(width='200px', height='40px'))

# Tạo nút để hiển thị biểu đồ dự đoán
pred_button = widgets.Button(description='Hiển thị biểu đồ dự đoán', layout=widgets.Layout(width='200px', height='40px'))

# Tạo nút đặt lại
reset_button = widgets.Button(description='Đặt lại', layout=widgets.Layout(width='200px', height='40px'))

# Kết nối nút hiển thị biểu đồ kỹ thuật với hàm
def on_tech_button_clicked(b):
    display_technical_chart(stock_dropdown.value)

# Kết nối nút hiển thị biểu đồ dự đoán với hàm
def on_pred_button_clicked(b):
    display_prediction_chart(stock_dropdown.value)

# Kết nối nút đặt lại
def on_reset_clicked(b):
    clear_output(wait=True)
    display(stock_dropdown, tech_button, pred_button, reset_button)

# Hiển thị widget ban đầu
display(stock_dropdown, tech_button, pred_button, reset_button)

# Kết nối các nút với sự kiện
tech_button.on_click(on_tech_button_clicked)
pred_button.on_click(on_pred_button_clicked)
reset_button.on_click(on_reset_clicked)